**1. Introducción**

El siguiente cuaderno realiza la multiplicación de 2 vectores utilizando GPGPU. El algoritmo representa una "similitud" con el algoritmo dot[4] con la diferencia que en el ejercicio el resultado obtenido termina siendo un vector y no un escalar.

La lógica planteada en dicho ejercicio es la siguiente: 

                      Y[0] = X[0] * Y[0]
                      Y[1] = X[1] * Y[1]
                      Y[2] = X[2] * Y[2]
                                .
                                .
                                .
                                .
                                .
                      Y[n] = X[n] * Y[n]


Siendo n, la cantidad de elementos ingresados por parámetro.

Realizada en lenguaje Python[1], utilizando Google Colab[2]

**2. Armado del ambiente**

Instala en el cuaderno el módulo CUDA de Python. [3]

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 9.0MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621228 sha256=b77a242437da0baf46ec8498c5549a12857fa8230b84dccadb09ddc04202ce56
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=593c94b039e8b283dffa9f83fcbf14b15a1177940d646ad0aa656e6fd91a1fe9
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


**3. Desarrollo**

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidad_N =   10000#@param {type: "number"}
# --------------------------------------------

from datetime import datetime

tiempo_total = datetime.now()

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

try:
  if cantidad_N <= 0:
    print("Debe ingresar un valor númerico mayor a 0")
  else:
    # CPU - Defino la memoria de los vectores en cpu.
    x_cpu = numpy.random.randn( cantidad_N )
    x_cpu = x_cpu.astype( numpy.float32() )

    y_cpu = numpy.random.randn( cantidad_N )
    y_cpu = y_cpu.astype( numpy.float32() )

    #tiempo_ini_cpu = datetime.now()

    r_cpu = numpy.empty_like( y_cpu )


    # CPU - reservo la memoria GPU.
    x_gpu = cuda.mem_alloc( x_cpu.nbytes )
    y_gpu = cuda.mem_alloc( y_cpu.nbytes )

    # GPU - Copio la memoria al GPU.
    cuda.memcpy_htod( x_gpu, x_cpu )
    cuda.memcpy_htod( y_gpu, y_cpu )

    # CPU - Defino la función kernel que ejecutará en GPU.
    module = SourceModule("""
    __global__ void kernel_prodVectorial( int n, float *X, float *Y)
    {
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
      if (idx < n)
      {
          Y[idx]  = X[idx] * Y[idx];
      }

    }
    """) 
    # CPU - Genero la función kernel.
    kernel = module.get_function("kernel_prodVectorial")

    tiempo_gpu = datetime.now()

    
    dim_hilo = 256
    dim_bloque = numpy.int( (cantidad_N+dim_hilo-1) / dim_hilo )
    
    # GPU - Ejecuta el kernel.
    kernel( numpy.int32(cantidad_N), x_gpu, y_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

    tiempo_gpu = datetime.now() - tiempo_gpu

    # GPU - Copio el resultado desde la memoria GPU.
    cuda.memcpy_dtoh( r_cpu, y_gpu )

    tiempo_total = datetime.now() - tiempo_total


    # CPU - Informo el resutlado.
    print( "------------------------------------")
    print( "Vector Resultante: " )
    print( r_cpu )
    print( "------------------------------------")

    print( "Cantidad de elementos: ", cantidad_N )
    print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
    print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )
    print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )
except Exception as exception:
  print("Ha ocurrido una excepcion: ", exception)

------------------------------------
Vector Resultante: 
[ 0.49734148 -0.01866157  0.13272288 ...  0.6960769  -0.36827222
  0.3486228 ]
------------------------------------
Cantidad de elementos:  10000
Thread x:  256 , Bloque x: 40
Tiempo Total:  2.794 [ms]
Tiempo GPU:  0.077 [ms]


**4. Tabla de pasos**

Procesador |	Función |	Detalle
-----------|----------|--------
CPU	       | @param	  | Lectura del tamaño de vectores desde Colab.
CPU	       | import	  | Importa los módulos para funcionar.
CPU	       | datetime.now()	| Toma el tiempo actual.
CPU        | if            | Verifico la cantidad ingresada
CPU	| numpy.random.randn( Cantidad_N )	| Inicializa los vectoes X e Y.
CPU | numpy.empty_like(y_cpu) | Devuelvo un nuevo vector con el mismo formato que el pasado por parametro
**GPU**	| cuda.mem_alloc() |	Reserva la memoria en GPU.
**GPU**	| cuda.memcpy_htod() | 	Copia las memorias desde el CPU al GPU.
CPU	| SourceModule()	| Define el código del kernel
CPU	| module.get_function()	| Genera la función del kernel GPU
CPU	| dim_tx/dim_bx	| Calcula las dimensiones.
**GPU**	| kernel()	| Ejecuta el kernel en GPU
CPU	| cuda.memcpy_dtoh( )	|  Copia el resultado desde GPU memoria Y a CPU memoria R.
CPU	|print()	| Informo el vector resultante, cantidad de elementos, etc.

**5. Conclusión**

Usando los mismos valores que en la versión CPU del ejercicio, se puede observar una clara mejora en los tiempos a niveles grandes de procesamiento. Recordando que en CPU utilizando 50.000 elemento duró un tiempo de 40,532 [ms], vemos que en GPU el tiempo es de 8,566 [ms]. Practicamente una 5ta parte del tiempo usando CPU. Y de ese tiempo, apenas 0,058 [ms] corresponden a GPU. 

Ahora bien, cuando la cantidad de elementos utilizada como prueba fue 10 ocurrió que el tiempo total del programa, fue mayor en GPU.

Tiempo Total CPU: 0,387 [ms]
Tiempo Total GPU: 2,087 [ms] - Tiempo GPU: 0,14 [ms]

Por lo tanto, se puede llegar a la conclusión de que es recomendable utilizar GPU siempre y cuando la cantidad de procesamiento sea grande (basandose en pruebas realizadas, la diferencia a favor del uso del GPU es a partir de 10.000 elementos)

**6. Bibliografía**

[1] Introducción a Python [Link](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)

[2] Introducción a Colab [Link](https://www.youtube.com/watch?v=ICJP_ukNSQ0)

[3] Documentación PYCUDA [Link](https://documen.tician.de/pycuda/)

[4] Algoritmo dot [Link](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-dot.html)
